In [6]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the labels for emotions
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

# Define the CNN model
def create_cnn_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),  # Specify input shape
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(emotion_labels), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Load and preprocess the dataset
def load_data():
    train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # For training and validation
    test_datagen = ImageDataGenerator(rescale=1./255)  # For testing

    train_data = train_datagen.flow_from_directory(
        'dataset/train',
        target_size=(48, 48),
        color_mode='grayscale',
        class_mode='categorical',
        batch_size=32,
        subset='training'
    )
    
    val_data = train_datagen.flow_from_directory(
        'dataset/train',
        target_size=(48, 48),
        color_mode='grayscale',
        class_mode='categorical',
        batch_size=32,
        subset='validation'
    )

    test_data = test_datagen.flow_from_directory(
        'dataset/test',
        target_size=(48, 48),
        color_mode='grayscale',
        class_mode='categorical',
        batch_size=32
    )
    
    return train_data, val_data, test_data

# Train the model
train_data, val_data, test_data = load_data()
input_shape = (48, 48, 1)
model = create_cnn_model(input_shape)
model.fit(train_data, epochs=10, validation_data=val_data)

# Save the trained model
model.save('emotion_cnn_model.keras')


Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Epoch 1/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 19s 23ms/step - accuracy: 0.2551 - loss: 1.8053 - val_accuracy: 0.3921 - val_loss: 1.5857
Epoch 2/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 16s 22ms/step - accuracy: 0.3904 - loss: 1.5568 - val_accuracy: 0.4572 - val_loss: 1.4097
Epoch 3/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 17s 24ms/step - accuracy: 0.4473 - loss: 1.4326 - val_accuracy: 0.4790 - val_loss: 1.3511
Epoch 4/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 22s 30ms/step - accuracy: 0.4899 - loss: 1.3272 - val_accuracy: 0.5132 - val_loss: 1.2876
Epoch 5/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 25s 35ms/step - accuracy: 0.5159 - loss: 1.2636 - val_accuracy: 0.5330 - val_loss: 1.2390
Epoch 6/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 25s 35ms/step - accuracy: 0.5370 - loss: 1.2120 - val_accuracy: 0.5187 - val_loss: 1.2515
Epoch 7/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 26s 36ms/step - accuracy: 0.5519 - loss: 1.1773 - val